<h3>Data Loading</h3>
<p>The Data Load feature supports loading a CSV file into an existing table in your database.  The data must be in the proper format to insert into the columns of your table.  Specifically, data types must match and any data constraints must be met.  No pre-processing of the data file is performed.  The maximum number of rows loaded is 100,000 if batch mode is selected.  The maximum number of rows is 500,000 if fast mode is selected.  Fast load will automatically be used for files with more than 100,000 rows in order to improve loading performance.  It can also be selected explicitly by the user. 
<br>   
<b>Note: The following limitations apply when fast load is used:</b>
    <li>The table must be empty</li>
    <li>The table cannot be a volatile table or global temporary table</li>
    <li>There must be sufficient room in the database to load the data</li>
<br>
    The status of your dataload is stored in the Teradata SQL History.  You can view the history by running the %history magic command.  If an error occurs while attempting to load your data, the entire load operation will be rolled back. 
<p>You can cancel the dataload once it has started by pressing the interrupt button on the notebook toolbar.
    
<p><b>***NOTE: To execute the SQL or magic command in the cell, press SHIFT+ENTER or the run button in the toolbar.</b>


#### Enter '%help dataload' to find out more about the %dataload magic

In [1]:
%help dataload

#### %dataload
Load a CSV file into a database table.

%dataload table=&lt;TableName&gt;, filepath=&lt;Path to CSV file&gt; [,database=&lt;DatabaseName&gt;][, batchsize=&lt;Rows per Batch&gt;] [, autocommit=&lt;True&gt;] [, mode=&lt;batch|fast&gt;] [, skiprows=&lt;rows to skip at the start of file&gt;]

* **database** is optional. if not specified, the default database of the current session is used.
* The **filepath** is relative to the JupyterLab workspace (where JupyterLab was launched).
* You can specify an absolute file path by starting the path with the OS specific directory notation.
* **batchsize** specifies the number of rows per batch.
  * Valid values are between 1 and 16383 for batch load. The default value is 5000 for batch load and 100000 for fast load.
* **autocommit** indicates whether to commit each batch load.
  * The default value is false (only commit when all batches have run). When autocommit is set to false and an error occurs, a rollback is performed.
  * When autocommit is set to true and an error occurs, any rows loaded will NOT be rolled back.
  * Set autocommit to true when a Database error is returned indicating the load has exceeded the maximum number of rowhash locks allowed.
  * autocommit is always turned off for fast load.
* **mode** specifies whether to use batch load or fast load. Fast load supports quick loading of large amounts of data.
  * It is recommended to use fast load only for large files with at least 100,000 rows.
  * The default mode is batch load for files with less than 100,000 rows. Fast load is used for large files with at least 100,000 rows.
  * The destination table must be empty to use fast load.
  * FastLoad cannot load into a volatile table or global temporary table.
  * Please make sure there is enough space in database before loading data. If you encounter "No more room in database" error using fast load, please make more space, drop and recreate the table, and then retry fast load.
  * For more information on fast load, please refer to github.com/Teradata/python-driver#fastload
* **skiprows** specifies number of rows to skip at start of the CSV file.
* The maximum number of rows that the **%dataload** magic will load is currently set to 100,000 rows for batch load and 500,000 for fast load.  Loading will stop when the maximum is reached.
* The data must be in the proper format to insert into the columns of your table. Specifically, data types must match and any data constraints must be met. 
No pre-processing of the data file is performed.
* If an error occurs while attempting to load your data and autocommit is false, the entire load operation will be rolled back. 
* The status of your dataload is stored in the Teradata SQL History.

#### Add a connection and connect to your Vantage system
Replace the systemName, user, host, and db values for your system.

In [ ]:
%var systemName=<Vantage-system>, user=<user name>, host=<host name or IP>, db=<database>

In [ ]:
%addconnect name=${systemName}, user=${user}, host=${host}

In [ ]:
%connect ${systemName}

#### Set your Vantage Database

In [ ]:
DATABASE ${db}

#### Create an empty table in your database

In [ ]:
CREATE MULTISET TABLE airports ,NO FALLBACK ,
     NO BEFORE JOURNAL,
     NO AFTER JOURNAL,
     CHECKSUM = DEFAULT,
     DEFAULT MERGEBLOCKRATIO
     (
      City VARCHAR(50) CHARACTER SET LATIN CASESPECIFIC,
      Name VARCHAR(50) CHARACTER SET LATIN CASESPECIFIC,
      Code VARCHAR(3) CHARACTER SET LATIN CASESPECIFIC)
PRIMARY INDEX ( City );

<h4>Run %dataload magic</h4>
<p>Run the %dataload magic, specifying the <b>DATABASE</b>, <b>TABLE</b>, and <b>FILEPATH</b> parameter values. The <b>FILEPATH</b> indicates the path to your CSV file. The path is relative to your JupyterLab workspace. You can also provide an absolute path to your CSV file in the <b>FILEPATH</b> value.
<p> Below shows loading the local airports.csv file (located in the notebooks/sql/data directory) into the airports table within your database.

In [ ]:
%dataload DATABASE=${db}, TABLE=airports, FILEPATH=notebooks/sql/data/airports.csv

<h4>Verify data was loaded by running SELECT query</h4>

In [ ]:
SELECT * FROM airports;

<h4>Data Load Status</h4>
<p>The status of the dataload is stored in the history. Run the %history magic to display the latest status.

In [ ]:
%history

#### Clean up table

In [ ]:
DROP TABLE airports;

Copyright 2021-2025 Teradata. All rights reserved.